In [1]:
# Dependencies
import modules.blast as blast
import modules.uniprot as uniprot
import modules.msa as msa
import pandas as pd
import numpy as np
import re
import time

In [2]:
# Constants
BLAST_CSV_PATH = 'data/blast.csv'  # Blast matches dataset
BLAST_FASTA_PATH = 'data/blast.fasta'  # Blast matched proteins fasta file
MSA_FASTA_PATH = 'data/msa.fasta'  # Multiple sequence alignment file
HUMAN_CSV_PATH = 'data/human.csv'  # Test set csv
HUMAN_IN_PFAM_CSV_PATH = 'data/human_in_PF00397.csv'
HUMAN_NO_PFAM_CSV_PATH = 'data/human_no_PF00397.csv'
HUMAN_FASTA_PATH = 'data/human.fasta'  # Test set fasta
GO_CSV_PATH = 'data/go.csv'  # GO terms dataset
FAMILIES_CSV_PATH = 'data/families.csv'  # Pfam families dataset

## BLAST search

Execute a blast search using EBI restful web service

In [3]:
# Start a BLAST job
status, job_id, _ = blast.run_job(
    email='damiano.clementel@studenti.unipd.it', sequence='VPSGWKAVFDDEYQTWYYVDLSTNSSQWEPP',
    database='uniref90', align=6
)

# Check output
print('Started job: {:s}'.format(job_id))

Started job: ncbiblast-R20200210-074203-0226-39139224-p2m


In [4]:
# Define results container
job_result = None
job_id = 'ncbiblast-R20200209-210205-0058-58696345-p2m'

# Retruieve BLAST job results
while True:
    # Make request for job status
    status, job_status, _ = blast.get_job_status(job_id)
    print('Job status: {:s}'.format(job_status))  # LOG
    # Check error
    if not status: break
    # Check if job has finished running
    if job_status == 'FINISHED':
        # Retrieve results
        status, job_result, _ = blast.get_job_result(job_id)
        print('Job exited with status: {:s}'.format(job_status))  # LOG
        break  # Exit loop
    # Wait 10 seconds before making another call
    time.sleep(10)

Job status: FINISHED
Job exited with status: FINISHED


In [5]:
# Job result is the tabular output as pandas DataFrame 
blast_match = job_result
# Remove database (it is kept in actual ac entry)
blast_match.subj_ac = blast_match.subj_ac.apply(lambda x: x.split(':')[1])
# Save dataset to disk
blast_match.to_csv(BLAST_CSV_PATH, index=False) 

# Show dataframe
blast_match.head()

,query_ac,subj_ac,perc_identity,align_len,mismatches,gap_opens,q_start,q_end,s_start,s_end,e_value,bit_score
0,EMBOSS_001,UniRef90_P43582,100.000,31,0,0,1,31,11,41,1.00e-15,74.7
1,EMBOSS_001,UniRef90_J8Q8J2,93.548,31,2,0,1,31,11,41,1.09e-14,72.0
2,EMBOSS_001,UniRef90_J5RH20,93.548,31,2,0,1,31,11,41,1.76e-14,71.2
3,EMBOSS_001,UniRef90_A0A0L8RJY2,93.548,31,2,0,1,31,11,41,4.72e-14,70.1
4,EMBOSS_001,UniRef90_A0A212M9M4,80.645,31,6,0,1,31,11,41,2.79e-12,65.5


## Create FASTA file from BLAST

In [6]:
# Save full fasta file
with open(BLAST_FASTA_PATH, 'w') as blast_fasta_file:
    # Get fasta file
    status, blast_fasta, _ = uniprot.map_ids(blast_match.subj_ac.tolist(), batch_size=300)
    # Write fasta file to disk
    blast_fasta_file.write(blast_fasta)

In [7]:
# Check number of retrieved fasta files
blast_fasta_len = sum([1 for c in blast_fasta if c == '>'])  # Number of rows in fasta file
blast_match_len = blast_match.shape[0]  # Number of rows in dataframe

assert  blast_fasta_len == blast_match_len, 'Fasta file and dataset lengths do not coincide'

## Execute multiple sequence alignment

In [8]:
# Make multiple sequence alignment

# Create MSA job
status, job_id, _ = msa.run_clustalo(email='damiano.clementel@studenti.unipd.it', sequence=blast_fasta)
# status, job_id, _ = msa.run_muscle(email='damiano.clementel@studenti.unipd.it', sequence=blast_fasta)

# Wait until MSA job finishes
while True:
    # Check response status
    status, job_status, _ = msa.get_job_status(job_id, algorithm=msa.CLUSTALO)
    # Check if response status is 200 OK
    if not status:
        print('Error: job exited')
        break 
    # Check if job has finished, then go on
    print('Job status: {:s}'.format(job_status))
    if job_status == 'FINISHED': break
    # Add delay
    time.sleep(3)
print()
    
# Get results
if status:
    # Retrieve result as fasta
    status, msa_fasta, _ = msa.get_job_result(job_id, algorithm=msa.CLUSTALO)
    # Save msa to file
    with open(MSA_FASTA_PATH, 'w') as msa_fasta_file:
        msa_fasta_file.write(msa_fasta)
    # Show fasta file head
    print('Retrieved fasta file')
    print(msa_fasta[:2000])

Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: FINISHED

Retrieved fasta file
>UniRef90_P43582 WW domain-containing protein WWM1 n=9 Tax=Saccharomyces TaxID=4930 RepID=WWM1_YEAST
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
-------

In [9]:
# Check length of retrieved fasta
msa_fasta_len = sum([1 for c in blast_fasta if c == '>'])  # Number of rows in fasta file

assert blast_fasta_len == blast_match_len, 'Alignments fasta file and dataset lengths do not coincide'

## Create human proteome dataset

In [10]:
# Retrieve dataset of human proteins
status, human_proteome, _ = uniprot.make_query(
    'reviewed:yes AND organism:"Homo sapiens (Human) [9606]"', 
    params={
        'compress': 'no',
        'columns': ','.join(['id', 'entry name', 'protein names', 'length', 
                             'go', 'database(PDB)', 'database(Pfam)']),
        'sort': 'score',
        'format': 'tab'
})

In [11]:
# Create DataFrame object

# Get rows and header row
rows = human_proteome.split('\n')
header = rows[0].split('\t')

# Instantiate new dataframe
human_proteome = pd.DataFrame([row.split('\t') for row in rows[1:] if row != ''], 
                              columns={'entry_ac': str(), 'entry_name': str(), 'protein_name': str(),
                                       'len': int(), 'go': str(), 'pdb_ids': str(), 'pfam_ids': str()})

# Save to disk
human_proteome.to_csv(HUMAN_CSV_PATH, sep='\t', index=False)
# Show first rows
human_proteome.head()

,entry_ac,entry_name,protein_name,len,go,pdb_ids,pfam_ids
0,Q9Y263,PLAP_HUMAN,Phospholipase A-2-activating protein (PLA2P) (...,795,cell [GO:0005623]; cell junction [GO:0030054];...,2K89;2K8A;2K8B;2K8C;3EBB;,PF09070;PF08324;PF00400;
1,Q96RE7,NACC1_HUMAN,Nucleus accumbens-associated protein 1 (NAC-1)...,527,cell junction [GO:0030054]; cytoplasm [GO:0005...,3GA1;4U2N;,PF10523;PF00651;
2,O43312,MTSS1_HUMAN,Protein MTSS 1 (Metastasis suppressor YGL-1) (...,755,actin cytoskeleton [GO:0015629]; cytoplasm [GO...,2D1K;,PF08397;PF02205;
3,Q9NP80,PLPL8_HUMAN,Calcium-independent phospholipase A2-gamma (EC...,782,endoplasmic reticulum membrane [GO:0005789]; G...,,PF01734;
4,Q15319,PO4F3_HUMAN,"POU domain, class 4, transcription factor 3 (B...",338,cytoplasm [GO:0005737]; nuclear chromatin [GO:...,,PF00046;PF00157;


In [12]:
# Add boolean column: is given protein in given PF00397 family?

# Define reference pfam
pfam_id = 'PF00397'
# Define matching pattern
pattern = r'(\;?){:s};'.format(pfam_id)

# Get families column
protein_families = human_proteome.pfam_ids
# Add "is in family?" column
human_proteome[pfam_id] = protein_families.apply(lambda x: re.search(pattern, str(x)) is not None)
human_proteome.to_csv(HUMAN_CSV_PATH, sep='\t', index=False)
human_proteome.head()

,entry_ac,entry_name,protein_name,len,go,pdb_ids,pfam_ids,PF00397
0,Q9Y263,PLAP_HUMAN,Phospholipase A-2-activating protein (PLA2P) (...,795,cell [GO:0005623]; cell junction [GO:0030054];...,2K89;2K8A;2K8B;2K8C;3EBB;,PF09070;PF08324;PF00400;,False
1,Q96RE7,NACC1_HUMAN,Nucleus accumbens-associated protein 1 (NAC-1)...,527,cell junction [GO:0030054]; cytoplasm [GO:0005...,3GA1;4U2N;,PF10523;PF00651;,False
2,O43312,MTSS1_HUMAN,Protein MTSS 1 (Metastasis suppressor YGL-1) (...,755,actin cytoskeleton [GO:0015629]; cytoplasm [GO...,2D1K;,PF08397;PF02205;,False
3,Q9NP80,PLPL8_HUMAN,Calcium-independent phospholipase A2-gamma (EC...,782,endoplasmic reticulum membrane [GO:0005789]; G...,,PF01734;,False
4,Q15319,PO4F3_HUMAN,"POU domain, class 4, transcription factor 3 (B...",338,cytoplasm [GO:0005737]; nuclear chromatin [GO:...,,PF00046;PF00157;,False


In [13]:
# Create protein family dataset

# Initialize protein family dataset
ds_families = {'entry_ac': [], 'pfam_id': []}

# Iterate through each human dataset row
for i, row in human_proteome.iterrows():
    # Get list of domain families for the current protein
    families = [f for f in str(row['pfam_ids']).split(';') if f != '']
    # Create a row for each family
    for j in range(len(families)):
        ds_families['entry_ac'].append(row['entry_ac'])
        ds_families['pfam_id'].append(families[j])
        
# Turn dataset into a Pandas DataFrame object
ds_families = pd.DataFrame(ds_families)
ds_families.to_csv(FAMILIES_CSV_PATH, sep='\t', index=False)  # Save to disk
ds_families.head()

,entry_ac,pfam_id
0,Q9Y263,PF09070
1,Q9Y263,PF08324
2,Q9Y263,PF00400
3,Q96RE7,PF10523
4,Q96RE7,PF00651


In [14]:
# Create gene ontology dataset

# Initialize gene ontology dataset
ds_gene_ontology = {'entry_ac': [], 'go_id': [], 'go_descr': []}

# Iterate through each human dataset row
for i, row in human_proteome.iterrows():
    entry = str(row['entry_ac'])
    go_str = str(row['go'])
    go_ids = re.findall(r'\[(GO:.+?)\]', go_str)
    go_descr = [re.sub(r'\[(GO:.+?)\]', '', s).strip() for s in re.split('; ', go_str)]
    # Create entries in new dataframe
    for j in range(len(go_ids)):
        ds_gene_ontology['entry_ac'].append(entry)
        go_id = re.sub('GO:', '', go_ids[j])
        ds_gene_ontology['go_id'].append(go_id)
        ds_gene_ontology['go_descr'].append(go_descr[j])
    
# Turn gene ontology dataset into Pandas DataFrame object
ds_gene_ontology = pd.DataFrame(ds_gene_ontology)
ds_gene_ontology.to_csv(GO_CSV_PATH, sep='\t', index=False)
ds_gene_ontology.head()

,entry_ac,go_id,go_descr
0,Q9Y263,0005623,cell
1,Q9Y263,0030054,cell junction
2,Q9Y263,0005737,cytoplasm
3,Q9Y263,0070062,extracellular exosome
4,Q9Y263,0005634,nucleus


### Human proteome in PF00397 family

In [15]:
# Get proteins in PF00397 family
human_in_pfam_ds = human_proteome[human_proteome['PF00397']]
human_in_pfam_ds.to_csv(HUMAN_IN_PFAM_CSV_PATH, sep='\t', index=False)
human_in_pfam_ds.head()

,entry_ac,entry_name,protein_name,len,go,pdb_ids,pfam_ids,PF00397
428,Q9BTA9,WAC_HUMAN,WW domain-containing adapter protein with coil...,647,nuclear speck [GO:0016607]; nucleoplasm [GO:00...,,PF00397;,True
871,Q9NZC7,WWOX_HUMAN,WW domain-containing oxidoreductase (EC 1.1.1....,414,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,1WMV;,PF00106;PF00397;,True
966,Q9GZV5,WWTR1_HUMAN,WW domain-containing transcription regulator p...,400,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,5N5R;5N5T;5N5W;5N75;,PF00397;,True
2246,Q8N3X1,FNBP4_HUMAN,Formin-binding protein 4 (Formin-binding prote...,1017,nuclear speck [GO:0016607],,PF00397;,True
2780,O15428,PINL_HUMAN,Putative PIN1-like protein (Peptidylprolyl cis...,100,cytosol [GO:0005829]; nucleus [GO:0005634]; pe...,,PF00397;,True


### Human proteome not in PF00397 family

In [16]:
# Get human proteins not in PF00397 family
human_no_pfam_ds = human_proteome[~human_proteome['PF00397']]
human_no_pfam_ds.to_csv(HUMAN_NO_PFAM_CSV_PATH, sep='\t', index=False)
human_no_pfam_ds.head()

,entry_ac,entry_name,protein_name,len,go,pdb_ids,pfam_ids,PF00397
0,Q9Y263,PLAP_HUMAN,Phospholipase A-2-activating protein (PLA2P) (...,795,cell [GO:0005623]; cell junction [GO:0030054];...,2K89;2K8A;2K8B;2K8C;3EBB;,PF09070;PF08324;PF00400;,False
1,Q96RE7,NACC1_HUMAN,Nucleus accumbens-associated protein 1 (NAC-1)...,527,cell junction [GO:0030054]; cytoplasm [GO:0005...,3GA1;4U2N;,PF10523;PF00651;,False
2,O43312,MTSS1_HUMAN,Protein MTSS 1 (Metastasis suppressor YGL-1) (...,755,actin cytoskeleton [GO:0015629]; cytoplasm [GO...,2D1K;,PF08397;PF02205;,False
3,Q9NP80,PLPL8_HUMAN,Calcium-independent phospholipase A2-gamma (EC...,782,endoplasmic reticulum membrane [GO:0005789]; G...,,PF01734;,False
4,Q15319,PO4F3_HUMAN,"POU domain, class 4, transcription factor 3 (B...",338,cytoplasm [GO:0005737]; nuclear chromatin [GO:...,,PF00046;PF00157;,False


### Retrieve and save FASTA formatted human proteome

In [17]:
# Save full fasta file
with open(HUMAN_FASTA_PATH, 'w') as human_fasta_file:
    # Get fasta file
    status, human_fasta, _ = uniprot.map_ids(human_proteome.entry_ac.tolist(),
                                             from_db='ACC', to_db='ACC', batch_size=300)
    # Write fasta file to disk
    human_fasta_file.write(human_fasta)

In [18]:
# Check the retrieved fasta file
print(human_fasta[:250])

>sp|Q9Y263|PLAP_HUMAN Phospholipase A-2-activating protein OS=Homo sapiens OX=9606 GN=PLAA PE=1 SV=2
MTSGATRYRLSCSLRGHELDVRGLVCCAYPPGAFVSVSRDRTTRLWAPDSPNRSFTEMHC
MSGHSNFVSCVCIIPSSDIYPHGLIATGGNDHNICIFSLDSPMPLYILKGHKNTVCSLSS
GKFGTLLSGSWDTTAKVWLNDKCMMTL
